In [1]:
# ! ln *py* ../prototype/ArcticAI_Prototype/validation_scripts/
! cd ../prototype/ArcticAI_Prototype/ && git add * */* && git commit -a -m "validation update" && git push


[master 898b2da] validation update
 Committer: Joshua J. Levy <f003k8w@gv01.hpcc.dartmouth.edu>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly:

    git config --global user.name "Your Name"
    git config --global user.email you@example.com

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 4 files changed, 40218 insertions(+), 350 deletions(-)
Counting objects: 5, done.
Delta compression using up to 32 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 60.74 KiB | 2.17 MiB/s, done.
Total 5 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/jlevy44/ArcticAI_Prototype
   c1bf4e4..898b2da  master -> master


In [1]:
import pickle
import os 
import sys, os
sys.path.insert(0,"/dartfs-hpc/rc/home/w/f003k8w/.local/lib/python3.7/site-packages/")
from sklearn.preprocessing import LabelEncoder
import torch
from torch_cluster import knn_graph
from torch_geometric.data import Data 
import numpy as np
from torch_geometric.utils import train_test_split_edges
import os
import argparse
from torch_geometric.utils.convert import to_networkx
from torch_geometric.data import InMemoryDataset,DataLoader
import os,glob, pandas as pd
from sklearn.metrics import f1_score,roc_auc_score
import copy
import torch
import torch.nn.functional as F
from collections import Counter
from torch import nn
from torch_geometric.nn import GCNConv, GATConv, DeepGraphInfomax, SAGEConv
from torch_geometric.nn import DenseGraphConv
from torch_geometric.utils import to_dense_batch, to_dense_adj, dense_to_sparse
from torch_geometric.nn import GINEConv
from torch_geometric.utils import dropout_adj
from torch_geometric.nn import APPNP
import torch.nn as nn
import fire
from visdom import Visdom
from torch_geometric.nn import GENConv, DeepGCNLayer
from sklearn.decomposition import PCA
from sklearn.utils.class_weight import compute_class_weight


EPS = 1e-15



def dense_mincut_pool(x, adj, s, mask=None):
    r"""MinCUt pooling operator from the `"Mincut Pooling in Graph Neural
    Networks" <https://arxiv.org/abs/1907.00481>`_ paper
    .. math::
        \mathbf{X}^{\prime} &= {\mathrm{softmax}(\mathbf{S})}^{\top} \cdot
        \mathbf{X}
        \mathbf{A}^{\prime} &= {\mathrm{softmax}(\mathbf{S})}^{\top} \cdot
        \mathbf{A} \cdot \mathrm{softmax}(\mathbf{S})
    based on dense learned assignments :math:`\mathbf{S} \in \mathbb{R}^{B
    \times N \times C}`.
    Returns pooled node feature matrix, coarsened symmetrically normalized
    adjacency matrix and two auxiliary objectives: (1) The minCUT loss
    .. math::
        \mathcal{L}_c = - \frac{\mathrm{Tr}(\mathbf{S}^{\top} \mathbf{A}
        \mathbf{S})} {\mathrm{Tr}(\mathbf{S}^{\top} \mathbf{D}
        \mathbf{S})}
    where :math:`\mathbf{D}` is the degree matrix, and (2) the orthogonality
    loss
    .. math::
        \mathcal{L}_o = {\left\| \frac{\mathbf{S}^{\top} \mathbf{S}}
        {{\|\mathbf{S}^{\top} \mathbf{S}\|}_F} -\frac{\mathbf{I}_C}{\sqrt{C}}
        \right\|}_F.
    Args:
        x (Tensor): Node feature tensor :math:`\mathbf{X} \in \mathbb{R}^{B
            \times N \times F}` with batch-size :math:`B`, (maximum)
            number of nodes :math:`N` for each graph, and feature dimension
            :math:`F`.
        adj (Tensor): Symmetrically normalized adjacency tensor
            :math:`\mathbf{A} \in \mathbb{R}^{B \times N \times N}`.
        s (Tensor): Assignment tensor :math:`\mathbf{S} \in \mathbb{R}^{B
            \times N \times C}` with number of clusters :math:`C`. The softmax
            does not have to be applied beforehand, since it is executed
            within this method.
        mask (BoolTensor, optional): Mask matrix
            :math:`\mathbf{M} \in {\{ 0, 1 \}}^{B \times N}` indicating
            the valid nodes for each graph. (default: :obj:`None`)
    :rtype: (:class:`Tensor`, :class:`Tensor`, :class:`Tensor`,
        :class:`Tensor`)
    """

    x = x.unsqueeze(0) if x.dim() == 2 else x
    adj = adj.unsqueeze(0) if adj.dim() == 2 else adj
    s = s.unsqueeze(0) if s.dim() == 2 else s

    (batch_size, num_nodes, _), k = x.size(), s.size(-1)

    s = torch.softmax(s, dim=-1)

    if mask is not None:
        mask = mask.view(batch_size, num_nodes, 1).to(x.dtype)
        x, s = x * mask, s * mask

    out = torch.matmul(s.transpose(1, 2), x)
    out_adj = torch.matmul(torch.matmul(s.transpose(1, 2), adj), s)

    # MinCUT regularization.
    mincut_num = _rank3_trace(out_adj)
    d_flat = torch.einsum('ijk->ij', adj)
    d = _rank3_diag(d_flat)
    mincut_den = _rank3_trace(
        torch.matmul(torch.matmul(s.transpose(1, 2), d), s))
    mincut_loss = -(mincut_num / mincut_den)
    mincut_loss = torch.mean(mincut_loss)

    # Orthogonality regularization.
    ss = torch.matmul(s.transpose(1, 2), s)
    i_s = torch.eye(k).type_as(ss)
    ortho_loss = torch.norm(
        ss / torch.norm(ss, dim=(-1, -2), keepdim=True,p=2) -
        i_s / torch.norm(i_s), dim=(-1, -2),p=2)
    ortho_loss = torch.mean(ortho_loss)

    # Fix and normalize coarsened adjacency matrix.
    ind = torch.arange(k, device=out_adj.device)
    out_adj[:, ind, ind] = 0
    d = torch.einsum('ijk->ij', out_adj)
    d = torch.sqrt(d)[:, None] + EPS
    out_adj = (out_adj / d) / d.transpose(1, 2)

    return out, out_adj, mincut_loss, ortho_loss


def _rank3_trace(x):
    return torch.einsum('ijj->i', x)


def _rank3_diag(x):
    eye = torch.eye(x.size(1)).type_as(x)
    out = eye * x.unsqueeze(2).expand(*x.size(), x.size(1))
    return out

class GCNNet(torch.nn.Module):
    def __init__(self, inp_dim, out_dim, hidden_topology=[32,64,128,128], p=0.5, p2=0.1, drop_each=True):
        super(GCNNet, self).__init__()
        self.out_dim=out_dim
        self.convs = nn.ModuleList([GATConv(inp_dim, hidden_topology[0])]+[GATConv(hidden_topology[i],hidden_topology[i+1]) for i in range(len(hidden_topology[:-1]))])
        self.drop_edge = lambda edge_index: dropout_adj(edge_index,p=p2)[0]
        self.dropout = nn.Dropout(p)
        self.fc = nn.Linear(hidden_topology[-1], out_dim)
        self.drop_each=drop_each

    def forward(self, x, edge_index, edge_attr=None):
        for conv in self.convs:
            if self.drop_each and self.training: edge_index=self.drop_edge(edge_index)
            x = F.relu(conv(x, edge_index, edge_attr))
        z=x
        if self.training:
            z = self.dropout(z)
        x = self.fc(z)
        return x,z
    
class DeeperGCN(torch.nn.Module):
    def __init__(self, n_input, n_output, hidden_channels, num_layers=28, edge_drop_p=0.05, p_dropout=0.1):
        super(DeeperGCN, self).__init__()

        self.node_encoder = nn.Linear(n_input, hidden_channels)
        self.p_dropout=p_dropout
#         self.edge_encoder = Linear(data.edge_attr.size(-1), hidden_channels)

        self.layers = torch.nn.ModuleList()
        self.drop_edge = lambda edge_index: dropout_adj(edge_index,p=edge_drop_p)[0]
        for i in range(1, num_layers + 1):
            conv = GENConv(hidden_channels, hidden_channels, aggr='softmax',
                           t=1.0, learn_t=True, num_layers=2, norm='layer')
            norm = nn.LayerNorm(hidden_channels, elementwise_affine=True)
            act = nn.ReLU(inplace=True)

            layer = DeepGCNLayer(conv, norm, act, block='res+', dropout=0.1,
                                 ckpt_grad=i % 3)
            self.layers.append(layer)
            

        self.lin = nn.Linear(hidden_channels, n_output)

    def forward(self, x, edge_index):
        x = self.node_encoder(x)
#         edge_attr = self.edge_encoder(edge_attr)

        x = self.layers[0].conv(x, edge_index, None)
        orig_edges=edge_index.shape[1]
        for layer in self.layers[1:]:
            x = layer(x, edge_index)
            edge_index=self.drop_edge(edge_index)
#         print(edge_index.shape[1]/orig_edges)

        z = self.layers[0].act(self.layers[0].norm(x))
        x = F.dropout(z, p=self.p_dropout, training=self.training)

        return self.lin(x),z
    
class GCNFeatures(torch.nn.Module):
    def __init__(self, gcn, bayes=False):
        super(GCNFeatures, self).__init__()
        self.gcn=gcn
        self.drop_each=bayes
    
    def forward(self, x, edge_index, edge_attr=None):
        for conv in self.gcn.convs:
            if self.drop_each and self.training: edge_index=self.gcn.drop_edge(edge_index)
            x = F.relu(conv(x, edge_index, edge_attr))
        if self.drop_each:
            x = self.gcn.dropout(x)
        y = F.softmax(self.gcn.fc(x))
        return x,y

def fit_model(
                model_basename="gnn_model.pth",
                log_basename="gnn_log.pth",
                graph_datasets=None,
                thumbnails=None,
                use_weights=False,
                n_batches_backward=1,
                f1_metric='weighted',
                n_epochs=1500,
                out_dir='gnn_models',
                lr=1e-2,
                eta_min=1e-4,
                T_max=20,
                wd=0,
                hidden_topology=[32,64,128,128],
                p=0.5,
                p2=0.3,
                burnin=400,
                warmup=100,
                gpu_id=0,
                batch_size=1,
                vis_every=0,
                port=5555,
                num_layers=28,
                use_thumbnails=False,
                use_deep=False, 
                save_every=100,
                tmp_save_file='gnn_models/tmp_model.pth'
                ):
    print(gpu_id); torch.cuda.set_device(gpu_id)
    if not vis_every: vis=None
    else: vis=Visdom(port=port)
    for k in graph_datasets:
        for i in range(len(graph_datasets[k])):
            graph_datasets[k][i].index=i
    train_dataset=graph_datasets['train']
    val_dataset=graph_datasets['val']
    print(len(train_dataset),len(val_dataset))
    y_true=np.hstack(graph.y_true.numpy() for graph in train_dataset)
    print(pd.Series(y_true).value_counts())
    y_true=y_true[y_true!=-1]
    weights=torch.tensor(compute_class_weight('balanced',np.unique(y_true),y_true))


    # load model
    if not use_deep:
        model=GCNNet(train_dataset[0].x.shape[1],
                     y_true.max()+1,
                     hidden_topology=hidden_topology,p=p,p2=p2)
    else:
        model=DeeperGCN(train_dataset[0].x.shape[1],
                     y_true.max()+1,
                     hidden_topology[0],edge_drop_p=p2,p_dropout=p,num_layers=num_layers)
    model=model.cuda()

    # load optimizer
    optimizer = torch.optim.Adam(model.parameters(),lr=lr,weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=eta_min, last_epoch=-1)
    criterion=nn.CrossEntropyLoss(weight=torch.tensor(weights).float() if use_weights else None)
    criterion=criterion.cuda()

    # initialize val saving
    save_mod=False
    past_performance=[0]
    best_model_dict=copy.deepcopy(model.state_dict())

    # dataloaders
    dataloaders={}

    dataloaders['train']=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
    dataloaders['val']=DataLoader(val_dataset,shuffle=True)
    dataloaders['warmup']=DataLoader(train_dataset,shuffle=False)
    train_loader=dataloaders['warmup']

    n_total_batches=0
    train_val_f1=[]
    for epoch in range(n_epochs):
        Y,Y_Pred=[],[]
        for i,data in enumerate(train_loader):
            n_total_batches+=1
            model.train(True)
            x=data.x.cuda()
            edge_index=data.edge_index.cuda()
            y=data.y_true.cuda()
            y_out, z=model(x,edge_index)
            loss = criterion(y_out[y!=-1], y[y!=-1]) / n_batches_backward
            loss.backward()
            if n_total_batches%n_batches_backward==0 or (i==len(train_loader.dataset)-1):
                optimizer.step()
                optimizer.zero_grad()
            Y_Pred.append(F.softmax(y_out).argmax(1)[y!=-1].detach().cpu().numpy().flatten())
            Y.append(y[y!=-1].detach().cpu().numpy().flatten())
#             if epoch%vis_every==0 and not i:
#                 vis.scatter(data.pos.numpy(),opts=dict(markercolor=(y_pred*255).astype(int),webgl=False,markerborderwidth=0,markersize=5,title="Train Predicted {}".format(data.id[0])),win="train_pred")
#                 vis.scatter(data.pos.numpy(),opts=dict(markercolor=y_true*255,webgl=False,markerborderwidth=0,markersize=5,title="Train Real {}".format(data.id[0])),win="train_true")
#                 vis.scatter(PCA(n_components=2).fit_transform(z.detach().cpu().numpy()),opts=dict(markercolor=y_true*255,webgl=False,markerborderwidth=0,markersize=5,title="Train PCA {}".format(data.id[0])),win="train_pca")
#                 vis.scatter(PCA(n_components=2).fit_transform(x.detach().cpu().numpy()),opts=dict(markercolor=y_true*255,webgl=False,markerborderwidth=0,markersize=5,title="Train Original PCA {}".format(data.id[0])),win="train_pca2")
#                 if use_thumbnails:
#                     vis.image(thumbnails['train'][data.index].transpose((2,1,0))[:,::-1,:],win="train_image")
            del x, edge_index, loss, y_out, z
            if epoch <=warmup:
                break 
        if epoch == warmup:
            train_loader=dataloaders['train']
        if epoch>=burnin:
            save_mod=True
        train_f1=f1_score(np.hstack(Y),np.hstack(Y_Pred),average=f1_metric)
        scheduler.step()
        Y,Y_Pred,Y_prob=[],[],[]
        for i,data in enumerate(dataloaders['val']):
            model.train(False)
            x=data.x.cuda()
            edge_index=data.edge_index.cuda()
            y=data.y_true.cuda()
            y_out,z=model(x,edge_index)
#             loss = criterion(y_out, y) 
            y_prob=F.softmax(y_out).detach().cpu().numpy()
            y_pred=y_prob.argmax(1).flatten()
            y_true=y.detach().cpu().numpy().flatten()
            Y_Pred.append(y_pred[y_true!=-1])
            Y.append(y_true[y_true!=-1])
            Y_prob.append(y_prob[y_true!=-1])
            if epoch%vis_every==0 and not i:
                vis.scatter(data.pos.numpy(),Y=y_pred+2,opts=dict(webgl=False,markerborderwidth=0,markersize=5,title="Val Predicted {}".format(data.id[0])),win="val_pred")
                vis.scatter(data.pos.numpy(),Y=y_true+2,opts=dict(webgl=False,markerborderwidth=0,markersize=5,title="Val Real {}".format(data.id[0])),win="val_true")
                vis.scatter(PCA(n_components=2).fit_transform(z.detach().cpu().numpy()),Y=y_true+2,opts=dict(webgl=False,markerborderwidth=0,markersize=5,title="Val PCA {}".format(data.id[0])),win="val_pca")
                vis.scatter(PCA(n_components=2).fit_transform(x.detach().cpu().numpy()),Y=y_true+2,opts=dict(webgl=False,markerborderwidth=0,markersize=5,title="Val Original PCA {}".format(data.id[0])),win="val_pca2")
                if use_thumbnails:
                    vis.image(thumbnails['val'][data.index].transpose((2,1,0))[:,::-1,:],win="val_image")
            del x, edge_index, y_out, z
        val_f1=f1_score(np.hstack(Y),np.hstack(Y_Pred),average=f1_metric)
        val_auc=roc_auc_score(np.hstack(Y),np.vstack(Y_prob),multi_class='ovr',average='macro')
        if save_mod and val_auc>=max(past_performance):
            best_model_dict=copy.deepcopy(model.state_dict())
            past_performance.append(val_auc)
        if save_mod and epoch % save_every==0:
            torch.save(model.state_dict(),tmp_save_file)
        print(epoch,train_f1,val_f1,val_auc,flush=True)
        train_val_f1.append((train_f1,val_f1,val_auc))
        if vis_every: vis.line(torch.tensor(np.array(train_val_f1)),win="f1")

    model.load_state_dict(best_model_dict)
    torch.save(model.state_dict(),os.path.join(out_dir,model_basename))
    torch.save(train_val_f1,os.path.join(out_dir,log_basename))
    

# class Commands(object):
#     def __init__(self):
#         pass
        
#     def fit_model(self,cv_split=2,
#                 graph_data='datasets/graph_dataset_pretrain.pkl',
#                 cv_splits='cv_splits/cv_splits.pkl',
#                 use_weights=False,
#                 n_batches_backward=1,
#                 f1_metric='weighted',
#                 n_epochs=1500,
#                 out_dir='models_no_pretrain',
#                 lr=1e-2,
#                 eta_min=1e-4,
#                 T_max=20,
#                 wd=0,
#                 hidden_topology=[32,64,128,128],
#                 p=0.5,
#                 p2=0.3,
#                 burnin=400,
#                 warmup=100,
#                 gpu_id=0,
#                 batch_size=1,
#                 vis_every=0,
#                 port=5555,
#                 num_layers=28,
#                 thumbnails="bcc/image_thumbnails.pkl",
#                 use_thumbnails=False,
#                 use_deep=False):
#         fit_model(cv_split,
#                 graph_data,
#                 cv_splits,
#                 use_weights,
#                 n_batches_backward,
#                 f1_metric,
#                 n_epochs,
#                 out_dir,
#                 lr,
#                 eta_min,
#                 T_max,
#                 wd,
#                 hidden_topology,
#                 p,
#                 p2,
#                 burnin,
#                 warmup,
#                 gpu_id,
#                 batch_size,
#                 vis_every,
#                 port,
#                 num_layers,
#                 thumbnails,
#                 use_thumbnails,
#                 use_deep)
#                 cv_split=2,
#                 graph_data='datasets/graph_dataset_no_pretrain_final.pkl',
#                 cv_splits='cv_splits/cv_splits_final.pkl',

In [2]:
import pandas as pd, os, glob
from functools import reduce
from operator import add
import os, tqdm
def check_if_exists(basename,analysis_type):
    return len(glob.glob(os.path.join("graph_datasets",analysis_type,f"{basename}*pkl")))>0
train_val_cases=pd.read_pickle("updated_patches/train_val_patch_info.pkl")

In [3]:
analysis_type='tumor'
cases={k:[basename for basename in train_val_cases[analysis_type][k]['ID'].unique() if check_if_exists(basename,analysis_type)] for k in ['train','val']}
graph_datasets={k:reduce(add,[pd.read_pickle(os.path.join('graph_datasets',analysis_type,f'{basename}.pkl')) for basename in tqdm.tqdm(cases[k],desc='case')]) for k in tqdm.tqdm(list(cases.keys()),desc='set')}
thumbnails={k:reduce(add,[pd.read_pickle(os.path.join('thumbnails',analysis_type,f'{basename}.pkl')) for basename in tqdm.tqdm(cases[k],desc='case')]) for k in tqdm.tqdm(list(cases.keys()),desc='set')}

set: 100%|██████████| 2/2 [00:01<00:00,  1.44it/s]


In [ ]:
# TOP VAL AUC: 0.97
fit_model(
                model_basename=f"{analysis_type}_gnn_model.pth",
                log_basename=f"{analysis_type}_gnn_log.pth",
                graph_datasets=graph_datasets,
                thumbnails=thumbnails,
                use_weights=True,
                n_batches_backward=1,
                f1_metric='weighted',
                n_epochs=815,
                out_dir='gnn_models',
                lr=1e-2,
                eta_min=1e-4,
                T_max=20,
                wd=0,
                hidden_topology=[32,64,64],
                p=0.5,
                p2=0.3,
                burnin=400,
                warmup=0,
                gpu_id=0,
                batch_size=6,
                vis_every=1,
                port=5556,
                num_layers=28,
                use_thumbnails=True,
                use_deep=False,
                tmp_save_file=f'gnn_models/{analysis_type}_gnn_model_tmp.pth',
                save_every=100
                )

Setting up a new session...
/home/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:243: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
/dartfs-hpc/rc/home/w/f003k8w/.local/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[0 1 2], y=[0 0 0 ... 0 0 0] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


0
225 105
0    624390
2    101325
1     23167
dtype: int64


/home/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:263: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:295: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:321: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:321: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0 0.17559015787363957 0.04307425991944918 0.6602923222540436


/home/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:295: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:321: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:321: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


...

/home/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:295: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:321: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:321: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


3 0.7100879950881093 0.754188678055903 0.8107664278988987


/home/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:295: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:321: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:321: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


4 0.7301563629490664 0.6516181389489505 0.8405076070172329


/home/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:295: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
analysis_type='macro'
cases={k:[basename for basename in train_val_cases[analysis_type][k]['ID'].unique() if check_if_exists(basename,analysis_type)] for k in ['train','val']}
graph_datasets={k:reduce(add,[pd.read_pickle(os.path.join('graph_datasets',analysis_type,f'{basename}.pkl')) for basename in tqdm.tqdm(cases[k],desc='case')]) for k in tqdm.tqdm(list(cases.keys()),desc='set')}
thumbnails={k:reduce(add,[pd.read_pickle(os.path.join('thumbnails',analysis_type,f'{basename}.pkl')) for basename in tqdm.tqdm(cases[k],desc='case')]) for k in tqdm.tqdm(list(cases.keys()),desc='set')}

In [ ]:
fit_model(
                model_basename=f"{analysis_type}_gnn_model.pth",
                log_basename=f"{analysis_type}_gnn_log.pth",
                graph_datasets=graph_datasets,
                thumbnails=thumbnails,
                use_weights=True,
                n_batches_backward=1,
                f1_metric='weighted',
                n_epochs=815,
                out_dir='gnn_models',
                lr=1e-2,
                eta_min=1e-4,
                T_max=20,
                wd=0,
                hidden_topology=[32,64,64],
                p=0.5,
                p2=0.3,
                burnin=400,
                warmup=0,
                gpu_id=0,
                batch_size=6,
                vis_every=1,
                port=5556,
                num_layers=28,
                use_thumbnails=True,
                use_deep=False,
                tmp_save_file=f'gnn_models/{analysis_type}_gnn_model_tmp.pth',
                save_every=100
                )